In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import requests
from bisect import bisect
import datetime

In [4]:
IHi_list = [50, 100, 200, 300, 400, 500]
ILo_list = [0, 51, 101, 201, 301, 401]
BPHi_list = [30, 60, 90, 120, 250, 500]
BPLo_list = [0, 31, 61, 91, 121, 251]

def pm25_aqi(Cp):
    index = bisect(BPLo_list, Cp) - 1
    aqi = ((IHi_list[index] - ILo_list[index]) / (BPHi_list[index] - BPLo_list[index])) * (Cp - BPLo_list[index]) + ILo_list[index]
    return aqi

In [5]:
def data_by_date(start_date, end_date, location_id):
    api_url = f"https://api.openaq.org/v2/measurements?date_from={start_date}&date_to={end_date}&limit=7000&sort=desc&parameter=pm25&location_id={location_id}&order_by=datetime"
    response = requests.get(api_url)
    print(response)

    results = response.json()["results"]
    df = pd.DataFrame(columns=['datetime', 'pm2.5'])

    for i, result in enumerate(results):
        datetime = result["date"]["local"]
        pm25 = result["value"]

        df.loc[i] = [datetime, pm25]

    df.datetime = pd.to_datetime(df.datetime)

    df['month'] = df['datetime'].dt.strftime('%m')
    df['day'] = df['datetime'].dt.strftime('%d')
    df['year'] = df['datetime'].dt.strftime('%Y')
    df['time'] = df['datetime'].dt.strftime('%H:%M:%S')

    df['sub_index'] = df['pm2.5'].map(pm25_aqi)

    return df

In [6]:
def month_wise_year_data(year, location_id):
    start_date = datetime.date(year, 1, 1).strftime('%Y-%m-%d')
    end_date = datetime.date(year, 12, 1).strftime('%Y-%m-%d')

    year_data = data_by_date(start_date, end_date, location_id)
    
    # return year_data.drop('pm2.5', axis=1).groupby(["day", "month", "year"]).mean().reset_index(level=[0, 1, 2]).pivot(index='day', columns=['month', 'year'])
    return year_data.drop(['pm2.5', 'year', 'datetime', 'time'], axis=1).pivot_table(values=['day', 'month'], index='day', columns=['month'], aggfunc=np.mean)['sub_index']

In [7]:
test = month_wise_year_data(2022, 6928)
test

<Response [200]>


month,02,03,04,05,06,07,08,09,10
day,,,,,,,,,
01,NaN,NaN,173.448736,NaN,27.390278,NaN,55.729492,NaN,37.381226
02,NaN,NaN,92.290345,45.766463,30.604861,NaN,74.155848,NaN,17.912779
03,72.391034,NaN,104.623376,40.637500,34.695833,NaN,48.662021,NaN,155.825703
04,291.158276,NaN,78.655973,47.361197,NaN,NaN,NaN,NaN,43.781609
05,138.466724,NaN,52.030936,93.040872,NaN,451.656742,NaN,NaN,87.213220
06,NaN,NaN,58.885057,105.247454,25.889744,61.650490,NaN,NaN,182.219094
07,NaN,NaN,59.028320,151.333576,19.391667,62.656384,NaN,NaN,74.775066
08,NaN,NaN,66.733733,141.383797,NaN,150.958432,NaN,NaN,51.848983
09,NaN,NaN,48.470848,44.016073,NaN,564.814739,NaN,NaN,14.766667
